In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import zipfile
import urllib.request
import os
import matplotlib.pyplot as plt
import shutil
from PIL import Image, ImageDraw
import random
import re
from scipy import spatial

In [ ]:
def normalize_posenet_vector(vecteur):
    
    x_components, y_components = zip(*vecteur)
    
    x_components, y_components = np.array(x_components), np.array(y_components)
    
    
    max_amplitude = max(max(x_components)-min(x_components),max(y_components)-min(y_components))
    
    amplification_factor = 1/max_amplitude
    
    x_components = amplification_factor*x_components
    y_components = amplification_factor*y_components
    
    def mean_extrema(array):
        return np.mean((max(array),min(array)))
    
    x_components, y_components =\
    x_components-mean_extrema(x_components) + 1/2, y_components-mean_extrema(y_components) + 1/2
    
    return list(zip(x_components, y_components))

In [ ]:
def parse_posenet_json(df, json_name, normalise_function = lambda x: x):
    
    with open(json_name,"r") as json:
        
        text = json.read()
        
        dics = []
        
        for item in eval(text):
            image_name = item.get("filename")
            
            image_id = int(re.match("\d+",image_name).group())
            
            poses = item.get("poses")
            
            normalized_poses = list(map(normalise_function, poses))
            
            dics.append({"image_identifier":image_id, "poses":poses, "norm_poses":normalized_poses})
    
    
    return df.merge(pd.DataFrame(dics), on = "image_identifier")

In [ ]:
full_paintings_df = pd.read_csv("data/partial_selected_full_paintings_df.csv")

In [ ]:
full_paintings_df = parse_posenet_json(full_paintings_df, "rijk_sample_expo/expo_rijk_posenet.json")\
.dropna(subset = ["poses"])

In [ ]:
full_paintings_df

In [ ]:
def show_pictures(poses, identifier, verbose = True):
    
    path = "rijk_sample_expo/{}.jpg".format(identifier)
    
    im = Image.open(path)
    
    if verbose:
        c = 0
        colors = ["red","yellow","green","blue","grey"]
        for pose in poses:

            color = colors[c]

            width, height = im.size

            draw = ImageDraw.Draw(im)

            pose = list(map(lambda tup : (width*tup[0], height*tup[1]), pose))

            for point in pose[:5]:
                x = point[0]
                y = point[1]
                r = 5
                draw.ellipse([x-r,y-r,x+r,y+r],color,color)

            for i in (5,6,11,12):
                draw.line([(pose[i][0],pose[i][1]),(pose[i+2][0],pose[i+2][1]),(pose[i+4][0],pose[i+4][1])],
                          fill = color, width = 5)

            c += 1
    
    plt.figure(figsize = (10,10))
    plt.imshow(np.asarray(im))
    plt.show()

In [ ]:
full_paintings_df.apply(lambda row: show_pictures(row["poses"],row["image_identifier"]), axis = 1)

In [ ]:
def compare_poses(pose1, pose2):
    return 1 - spatial.distance.cosine(np.array(pose1).flatten(),np.array(pose2).flatten())

def max_similarity(pose1, list_poses):
    return max(map(lambda p: compare_poses(pose1, p), list_poses))

# Show N best matchs for 3 test pictures

In [ ]:
N_RESULTS_TO_SHOW = 5

with open("rijk_sample_expo/inputs/inputs.json") as input_json:
    
    poses_dicts = eval(input_json.read())
    
    for filename in os.listdir("rijk_sample_expo/inputs"):
        if filename.endswith("jpg"):
            im = Image.open("rijk_sample_expo/inputs/{}".format(filename))

            pose1 = None
            for item in poses_dicts:
                if item.get("filename") == filename:
                    pose1 = normalize_posenet_vector(item.get("poses")[0])
            
            similarities = full_paintings_df["norm_poses"].apply(lambda poses: max_similarity(pose1, poses))

            ranked_df = full_paintings_df.join(similarities, lsuffix='_caller', rsuffix='_other')\
            .sort_values(by = "norm_poses_other", ascending = False)
            
            plt.figure(figsize = (10,10))
            plt.imshow(np.asarray(im))
            plt.show()
            
            for _, row in ranked_df.head(N_RESULTS_TO_SHOW).iterrows():
                show_pictures(row["poses"],row["image_identifier"], verbose = False)

            